In [ ]:
]pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [46]:
import os
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry


In [47]:
def configurar_cliente_openmeteo():
    """Configura o cliente da Open-Meteo com cache e retry."""
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    return openmeteo_requests.Client(session=retry_session)

In [48]:
def consultar_dados_climaticos(latitude, longitude, data_inicio, data_fim, variaveis_diarias, cliente):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": data_inicio,
        "end_date": data_fim,
        "daily": variaveis_diarias
    }
    responses = cliente.weather_api(url, params=params)
    return responses[0]

In [49]:
def processar_dados_diarios(response, nomes_variaveis):
    daily = response.Daily()
    datas = pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left"
    )

    dados = {"date": datas}
    for i, nome in enumerate(nomes_variaveis):
        valores = daily.Variables(i).ValuesAsNumpy()
        dados[nome] = valores

    return pd.DataFrame(data=dados)


In [50]:
def salvar_csv_municipio(df, pais, municipio, latitude, longitude, pasta_destino):
    # Adiciona colunas extras
    df["pais"] = pais
    df["municipio"] = municipio
    df["latitude"] = latitude
    df["longitude"] = longitude

    # Gera nome do arquivo
    nome_arquivo = f"{pais}_{municipio}_clima.csv".replace(" ", "_")
    caminho = os.path.join(pasta_destino, nome_arquivo)

    # Cria a pasta se não existir
    os.makedirs(pasta_destino, exist_ok=True)

    df.to_csv(caminho, index=False)
    print(f"✔ Arquivo salvo: {caminho}")


In [51]:
def executar_em_lote(lista_municipios, data_inicio, data_fim, variaveis_diarias, pasta_destino):
    cliente = configurar_cliente_openmeteo()

    for entrada in lista_municipios:
        pais = entrada["pais"]
        municipio = entrada["municipio"]
        lat = entrada["latitude"]
        lon = entrada["longitude"]

        try:
            print(f"🔄 Processando: {pais} - {municipio}")
            resposta = consultar_dados_climaticos(lat, lon, data_inicio, data_fim, variaveis_diarias, cliente)
            df = processar_dados_diarios(resposta, variaveis_diarias)
            salvar_csv_municipio(df, pais, municipio, lat, lon, pasta_destino)
        except Exception as e:
            print(f"❌ Erro ao processar {municipio}: {e}")


In [53]:

# ------------------------------
# EXEMPLO DE USO
# ------------------------------
lista_municipios = [
        {"pais": "Brasil", "municipio": "Marilia", "latitude": -22.2139, "longitude": -49.9458},
        {"pais": "Portugal", "municipio": "Lisboa", "latitude": 38.7169, "longitude": -9.1399},
        {"pais": "Portugal", "municipio": "Porto", "latitude": 41.1496, "longitude": -8.6109},
    ]

variaveis_diarias = [
        "temperature_2m_mean", "temperature_2m_max", "temperature_2m_min",
        "apparent_temperature_mean", "apparent_temperature_max", "apparent_temperature_min",
        "precipitation_sum", "windspeed_10m_max"
    ]

data_inicio = "2024-07-01"
data_fim = "2025-06-30"
pasta_destino = "..\\..\\Bases\\Clima\\openMeteoApi"

executar_em_lote(lista_municipios, data_inicio, data_fim, variaveis_diarias, pasta_destino)

🔄 Processando: Brasil - Marilia
✔ Arquivo salvo: ..\..\Bases\Clima\openMeteoApi\Brasil_Marilia_clima.csv
🔄 Processando: Portugal - Lisboa
✔ Arquivo salvo: ..\..\Bases\Clima\openMeteoApi\Portugal_Lisboa_clima.csv
🔄 Processando: Portugal - Porto
✔ Arquivo salvo: ..\..\Bases\Clima\openMeteoApi\Portugal_Porto_clima.csv
